In [15]:
!pip install -q scikit-optimize

     |████████████████████████████████| 100 kB 3.7 MB/s 


XGBRegressor hyperparameter optimization based on bayesian approach

In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/kaggle/house_pricing'

/content/drive/MyDrive/Colab Notebooks/kaggle/house_pricing


In [19]:
from common import *
from xgboost import XGBRegressor
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize

In [5]:
x_train, y_train, x_test = read_preprocessed_data(postfix='preprocessed')
print(x_train.shape, y_train.shape, x_test.shape)

(1451, 309) (1451, 1) (1459, 309)


In [24]:
def params_to_space(params):
    space = []
    for key in params.keys():
        value = params[key]
        if (type(value) == range):
            space.append(Integer(min(value), max(value), name=key))
        elif (type(value) == list):
            if (type(value[0]) == int):
                space.append(Integer(min(value), max(value), name=key))
            else:
                space.append(Real(min(value), max(value), name=key))
    return space
  

def hyperopt_model(x, y, model, space, n_iter=50, history={}):

    @use_named_args(space)
    def objective(**params):
        if ('iter' not in history):
            history['iter'] = 0
            history['err'] = 1e9

        model.set_params(**params)
        err = get_cv_score(x, y, model)

        history['iter'] += 1
        if (err < history['err']):
            history['err'] = err
            history['params'] = params

        print(datetime.now())
        print(history['iter'], 'of', n_iter, 'err:', err, 'params:', params)
        print('best', history['err'], history['params'])
        print('')
        return err

    results = gp_minimize(objective, space, n_calls=n_iter)
    history.clear()
    return (results.fun, results.x)

In [ ]:
params = {
  'max_depth': [2, 3, 4, 5, 6, 7],
  'n_estimators': range(500, 3600),
  'learning_rate': np.hstack((np.linspace(0.001, 0.01, 10), 
                              np.linspace(0.02, 0.1, 9), [0.2, 0.3])),
  'min_child_weight': range(0, 10), 
  'subsample': np.linspace(0.1, 1, 10),
  'colsample_bytree': np.linspace(0.1, 1, 10),
  'gamma': range(0, 200),
  'lambda': range(0, 200),
  'alpha': range(0, 200)
}
space = params_to_space(params)
model = XGBRegressor(objective='reg:squarederror')
hyperopt_model(x_train, y_train, model, space, n_iter=10000)

2022-01-18 19:20:46.832835
15 of 10000 err: 15209.045726926324 params: {'max_depth': 2, 'n_estimators': 1876, 'min_child_weight': 1, 'gamma': 73, 'lambda': 48, 'alpha': 106}
best 15209.045726926324 {'max_depth': 2, 'n_estimators': 1876, 'min_child_weight': 1, 'gamma': 73, 'lambda': 48, 'alpha': 106}

2022-01-18 19:21:17.049334
16 of 10000 err: 15917.256678035017 params: {'max_depth': 3, 'n_estimators': 698, 'min_child_weight': 6, 'gamma': 176, 'lambda': 148, 'alpha': 84}
best 15209.045726926324 {'max_depth': 2, 'n_estimators': 1876, 'min_child_weight': 1, 'gamma': 73, 'lambda': 48, 'alpha': 106}

2022-01-18 19:23:06.682166
17 of 10000 err: 15991.92236578312 params: {'max_depth': 3, 'n_estimators': 2579, 'min_child_weight': 7, 'gamma': 174, 'lambda': 66, 'alpha': 89}
best 15209.045726926324 {'max_depth': 2, 'n_estimators': 1876, 'min_child_weight': 1, 'gamma': 73, 'lambda': 48, 'alpha': 106}



Conclusion: according to experimental results, this approach works worse than randomized grid search.